In [5]:
import functools

import numpy as np

from pycobyla.gcobyla import GCobyla

import tests.test_custom as tc

# Gaussian simplex example

In [6]:
def gaussian_optimizer():
    F = functools.partial(tc.gaussian, A=-1)
    c1 = lambda x: 1 - x[0]
    c2 = lambda x: 1 + x[0]
    c3 = lambda x: 1 - x[1]
    c4 = lambda x: 1 + x[1]

    C = (c1, c2, c3, c4)
    x = np.ones(2)    
    mu = np.array((0, 0))
    
    opt = GCobyla(x, F, C, rhobeg=0.5, rhoend=1e-12)
    return opt

In [7]:
%matplotlib widget
#%matplotlib notebook
import itertools

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np


def simplex_3d_plot(opt):
    plt.close('all')
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    
    points = np.array((opt.optimal_vertex, *(opt.sim + opt.optimal_vertex), opt.optimal_vertex))
    points = np.c_[points, np.zeros(points.shape[0])]
    ax.plot(points[0,0], points[0,1], points[0, 2], 'go')
    ax.plot(points[1:-1,0], points[1:-1,1], points[1:-1, 2], 'bo')
    ax.plot(points[...,0], points[...,1], points[..., 2], 'r-', lw=2)
    
    # Make data.
    max_p = points.max(axis=0)[:-1].max()
    min_p = points.min(axis=0)[:-1].min()

    X = np.linspace(min_p, max_p, 50)
    xv, yv = np.meshgrid(X, X)
    zv = np.array(tuple(opt.F(xy) for xy in itertools.product(X, X)))
    zv = zv.reshape(xv.shape)

    # Plot the surface.
    surf = ax.plot_surface(xv, yv, zv, cmap=cm.plasma, linewidth=0, antialiased=False)

    # Customize the z axis.
    ax.set_zlim(-1., 0.1)
    ax.zaxis.set_major_locator(LinearLocator(10))
    ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

    plt.show()

In [8]:
opt = gaussian_optimizer()
steps_it =(step for step in opt.g_run() if step in {GCobyla.BEFORE_REVIEW_CURRENT_SIMPLEX_CHECKPOINT, GCobyla.BEFORE_GENERATE_X_START_CHECKPOINT})

simplex_3d_plot(opt)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
next(steps_it)
simplex_3d_plot(opt)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
def simplex_2d_plot(opt, target):
    plt.close('all')
    fig, ax = plt.subplots()
    
    points = np.array((opt.optimal_vertex, *(opt.sim + opt.optimal_vertex), opt.optimal_vertex))
    ax.plot(*opt.optimal_vertex, color='limegreen', marker='o')
    ax.plot(points[1:-1,0], points[1:-1,1], '.', color='blue', marker='o')
    ax.plot(points[...,0], points[...,1], '-.', color='red', lw=2)
    ax.plot(target[0], target[1], color='limegreen', marker='*')
    
    # Make data
    all_points = np.array((*points, target))
    max_p = all_points.max(axis=0).max()
    min_p = all_points.min(axis=0).min()
    diff_p = max_p - min_p

    X = np.linspace(max_p + (0.1 * diff_p), min_p - (0.1 * diff_p), 50)
    xv, yv = np.meshgrid(X, X)
    zv = np.array(tuple(-opt.F(xy) for xy in itertools.product(X, X)))
    zv = zv.reshape(xv.shape)
    
    cs = ax.contourf(xv, yv, zv, cmap=cm.magma)
    cbar = fig.colorbar(cs)
    
    plt.show()

In [11]:
opt = gaussian_optimizer()
steps_it = (step for step in opt.g_run() if step in {GCobyla.BEFORE_REVIEW_CURRENT_SIMPLEX_CHECKPOINT, GCobyla.BEFORE_GENERATE_X_START_CHECKPOINT})

simplex_2d_plot(opt, target=(0,0))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
next(steps_it)
simplex_2d_plot(opt, target=(0,0))
opt.data

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

nfvals: 3
x: [1. 1.]
optimal_vertex: [1. 1.]
datmat: 
[[-0.5         2.5         0.          2.         -0.19691168  0.5       ]
 [ 0.          2.         -0.5         2.5        -0.19691168  0.5       ]
 [ 0.          2.          0.          2.         -0.36787944  0.        ]]
a: 
None
sim: 
[[0.5 0. ]
 [0.  0.5]]
simi: 
[[2. 0.]
 [0. 2.]]
optimal vertex: 
[1. 1.]
parmu: 0
rho: 0.5


In [13]:
for _ in steps_it: pass
simplex_2d_plot(opt, target=(0,0))
opt.data

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

nfvals: 88
x: [1.40230486e-10 8.95344630e-11]
optimal_vertex: [1.40230486e-10 8.95344630e-11]
datmat: 
[[ 1.  1.  1.  1. -1.  0.]
 [ 1.  1.  1.  1. -1.  0.]
 [ 1.  1.  1.  1. -1.  0.]]
a: 
[[-1.00000002e+00 -1.16238286e-08]
 [ 9.99999908e-01  5.78209700e-09]
 [-7.92720677e-08 -9.99999987e-01]
 [-2.66223273e-08  1.00000001e+00]
 [-0.00000000e+00 -0.00000000e+00]]
sim: 
[[-4.88351701e-13  1.07296861e-13]
 [-3.90343708e-13 -1.77661314e-12]]
simi: 
[[-1.95340680e+12 -1.17974147e+11]
 [ 4.29187445e+11 -5.36948373e+11]]
optimal vertex: 
[1.40230486e-10 8.95344630e-11]
parmu: 0
rho: 1e-12


# Gaussian track examples 

In [14]:
%matplotlib widget

from tests.nb.utils import plot_track

In [15]:
opt = gaussian_optimizer()
steps_it = (step for step in opt.g_run() if step in {GCobyla.BEFORE_REVIEW_CURRENT_SIMPLEX_CHECKPOINT, GCobyla.BEFORE_GENERATE_X_START_CHECKPOINT})

In [16]:
next(steps_it)
plot_track(opt, target=np.zeros(2), n_points=5, plot_simplex=True, plot_trure=True, aspect=True)
opt.data

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

nfvals: 3
x: [1. 1.]
optimal_vertex: [1. 1.]
datmat: 
[[-0.5         2.5         0.          2.         -0.19691168  0.5       ]
 [ 0.          2.         -0.5         2.5        -0.19691168  0.5       ]
 [ 0.          2.          0.          2.         -0.36787944  0.        ]]
a: 
None
sim: 
[[0.5 0. ]
 [0.  0.5]]
simi: 
[[2. 0.]
 [0. 2.]]
optimal vertex: 
[1. 1.]
parmu: 0
rho: 0.5


In [17]:
for _ in steps_it: pass
plot_track(opt, target=np.zeros(2), n_points=5, plot_simplex=True, plot_trure=True, aspect=True)
opt.data
print(opt.sim / opt.sim.min())
print(opt.sim @ opt.sim.T)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

nfvals: 88
x: [1.40230486e-10 8.95344630e-11]
optimal_vertex: [1.40230486e-10 8.95344630e-11]
datmat: 
[[ 1.  1.  1.  1. -1.  0.]
 [ 1.  1.  1.  1. -1.  0.]
 [ 1.  1.  1.  1. -1.  0.]]
a: 
[[-1.00000002e+00 -1.16238286e-08]
 [ 9.99999908e-01  5.78209700e-09]
 [-7.92720677e-08 -9.99999987e-01]
 [-2.66223273e-08  1.00000001e+00]
 [-0.00000000e+00 -0.00000000e+00]]
sim: 
[[-4.88351701e-13  1.07296861e-13]
 [-3.90343708e-13 -1.77661314e-12]]
simi: 
[[-1.95340680e+12 -1.17974147e+11]
 [ 4.29187445e+11 -5.36948373e+11]]
optimal vertex: 
[1.40230486e-10 8.95344630e-11]
parmu: 0
rho: 1e-12
[[ 0.27487791 -0.06039405]
 [ 0.21971227  1.        ]]
[[2.50000000e-25 0.00000000e+00]
 [0.00000000e+00 3.30872245e-24]]


# Pyramid track example: no convergence due to bad rhobeg

In [18]:
%matplotlib widget
import functools

import tests.test_custom as tc
from tests.nb.utils import plot_track


def pyramid_optimizer(start_x=None, width=2, rhobeg=.05, rhoend=1e-12):
    F = functools.partial(tc.pyramid, center=np.zeros(2), width=width, height=-1)
    C = ()
    start_x = np.random.uniform(low=-width/2, high=width/2, size=2) if start_x is None else start_x
    opt = GCobyla(start_x, F, C, rhobeg=rhobeg, rhoend=rhoend)
    return opt

In [19]:
start_x = np.array((-0.57714299498416465894479, -0.89532481179992373654386))
opt = pyramid_optimizer(start_x=start_x, rhobeg=.05)
steps_it = (step for step in opt.g_run() if step in {GCobyla.BEFORE_REVIEW_CURRENT_SIMPLEX_CHECKPOINT, GCobyla.BEFORE_GENERATE_X_START_CHECKPOINT})

In [20]:
next(steps_it)
plot_track(opt, target=np.zeros(2), n_points=5, plot_simplex=False, plot_trure=False, aspect=True)
opt.data

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

nfvals: 3
x: [-0.57714299 -0.84532481]
optimal_vertex: [-0.57714299 -0.84532481]
datmat: 
[[-0.10467519  0.        ]
 [-0.10467519  0.        ]
 [-0.15467519  0.        ]]
a: 
None
sim: 
[[ 0.05 -0.05]
 [ 0.   -0.05]]
simi: 
[[ 20. -20.]
 [  0. -20.]]
optimal vertex: 
[-0.57714299 -0.84532481]
parmu: 0
rho: 0.05


In [21]:
for _ in steps_it: pass
fig, (ax1, ax2) = plot_track(opt, target=np.zeros(2), n_points=5, plot_simplex=True, plot_trure=True, aspect=True)
ax2.plot(np.array((-1, 1)), np.array((1, -1)), linestyle='-.')
ax2.plot(np.array((-1, 1)), np.array((-1, 1)), linestyle='-.')

margin = 1e-1
xmin = min((*opt.track[:, 0], 0))
xmax = max((*opt.track[:, 0], 0))
ymin = min((*opt.track[:, 1], 0))
ymax = max((*opt.track[:, 1], 0))
ax2.set(xlim=(xmin - margin, xmax + margin), ylim=(ymin - margin, ymax + margin))

opt.data

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

nfvals: 418
x: [-0.10295554 -0.10295554]
optimal_vertex: [-0.10295554 -0.10295554]
datmat: 
[[-0.89704446  0.        ]
 [-0.89704446  0.        ]
 [-0.89704446  0.        ]]
a: 
[[0.00122365 0.98743908]]
sim: 
[[-9.95288611e-13 -1.09695682e-12]
 [ 1.23921648e-15  9.99999232e-13]]
simi: 
[[-1.00610784e+12 -1.10365770e+12]
 [ 1.24678637e+09  1.00136844e+12]]
optimal vertex: 
[-0.10295554 -0.10295554]
parmu: 0
rho: 1e-12


### Convergency with a suitable rhobeg

In [22]:
opt = pyramid_optimizer(start_x=start_x, rhobeg=.1)
steps_it = (step for step in opt.g_run() if step in {GCobyla.BEFORE_REVIEW_CURRENT_SIMPLEX_CHECKPOINT, GCobyla.BEFORE_GENERATE_X_START_CHECKPOINT})

for _ in steps_it: pass
fig, (ax1, ax2) = plot_track(opt, target=np.zeros(2), n_points=5, plot_simplex=True, plot_trure=True, aspect=True)
ax2.plot(np.array((-1, 1)), np.array((1, -1)), linestyle='-.')
ax2.plot(np.array((-1, 1)), np.array((-1, 1)), linestyle='-.')

margin = 1e-1
xmin = min((*opt.track[:, 0], 0))
xmax = max((*opt.track[:, 0], 0))
ymin = min((*opt.track[:, 1], 0))
ymax = max((*opt.track[:, 1], 0))
_ = ax2.set(xlim=(xmin - margin, xmax + margin), ylim=(ymin - margin, ymax + margin))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# 4 faces pyramid track example: no convergence due to bad rhobeg

In [23]:
%matplotlib widget

import functools
import numpy as np

import tests.test_custom as tc
from tests.nb.utils import plot_track


def pyramid_faces_optimizer(start_x, radius=2, faces=4, rhobeg=.5, rhoend=1e-12):
    F = functools.partial(tc.pyramid_faces, center=np.zeros(2), radius=radius, height=-1, faces=faces)
    c1 = lambda x: 1 - sum(x ** 2)
    C = (c1,)
    opt = GCobyla(start_x, F, C, rhobeg=rhobeg, rhoend=rhoend)
    return opt

In [24]:
start_x = np.array((0.60522848317453536992616, -0.92243950619371162247262))
opt = pyramid_faces_optimizer(start_x=start_x, faces=4, rhobeg=.5)
steps_it = (step for step in opt.g_run() if step in {GCobyla.BEFORE_REVIEW_CURRENT_SIMPLEX_CHECKPOINT, GCobyla.BEFORE_GENERATE_X_START_CHECKPOINT})

In [25]:
next(steps_it)
fig, (ax1, ax2) = plot_track(opt, target=np.zeros(2), n_points=5, plot_simplex=False, plot_trure=False, aspect=False)
ax2.plot(np.array((-1, 1)), np.array((0, 0)), linestyle='-.')
ax2.plot(np.array((0, 0)), np.array((-1, 1)), linestyle='-.')

margin = 1e-2
xmin = min((*opt.track[:, 0], 0))
xmax = max((*opt.track[:, 0], 0))
ymin = min((*opt.track[:, 1], 0))
ymax = max((*opt.track[:, 1], 0))
ax2.set(xlim=(xmin - margin, xmax + margin), ylim=(ymin - margin, ymax + margin))
opt.data

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

nfvals: 3
x: [ 0.60522848 -0.42243951]
optimal_vertex: [ 0.60522848 -0.42243951]
datmat: 
[[-1.07242464  0.          1.07242464]
 [-0.21719616 -0.23616601  0.21719616]
 [ 0.45524335 -0.48616601  0.        ]]
a: 
None
sim: 
[[ 0.5 -0.5]
 [ 0.  -0.5]]
simi: 
[[ 2. -2.]
 [ 0. -2.]]
optimal vertex: 
[ 0.60522848 -0.42243951]
parmu: 0
rho: 0.5


In [26]:
for _ in steps_it: pass
fig, (ax1, ax2) = plot_track(opt, target=np.zeros(2), n_points=5, plot_simplex=True, plot_trure=True)
ax2.plot(np.array((-1, 1)), np.array((0, 0)), linestyle='-.')
ax2.plot(np.array((0, 0)), np.array((-1, 1)), linestyle='-.')

margin = 1e-1
xmin = min((*opt.track[:, 0], 0))
xmax = max((*opt.track[:, 0], 0))
ymin = min((*opt.track[:, 1], 0))
ymax = max((*opt.track[:, 1], 0))
ax2.set(xlim=(xmin - margin, xmax + margin), ylim=(ymin - margin, ymax + margin))

opt.data
print(opt.F(opt.optimal_vertex))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

nfvals: 371
x: [8.34524795e-02 7.67764511e-13]
optimal_vertex: [8.34524795e-02 1.02106460e-13]
datmat: 
[[ 0.99303568 -0.95827376  0.        ]
 [ 0.99303568 -0.95827376  0.        ]
 [ 0.99303568 -0.95827376  0.        ]]
a: 
[[-1.66904975e-01 -1.01481399e-09]
 [-4.52220685e-01  4.03378970e-01]]
sim: 
[[-7.46256899e-13  6.65658050e-13]
 [ 1.60349663e-12 -2.63840311e-13]]
simi: 
[[3.03094787e+11 7.64695447e+11]
 [1.84206677e+12 8.57285889e+11]]
optimal vertex: 
[8.34524795e-02 1.02106460e-13]
parmu: 0
rho: 1e-12
-0.95827376022649117456


### Convergency with a suitable rhobeg

In [27]:
opt = pyramid_faces_optimizer(start_x=start_x, faces=4, rhobeg=.1)
steps_it = (step for step in opt.g_run() if step in {GCobyla.BEFORE_REVIEW_CURRENT_SIMPLEX_CHECKPOINT, GCobyla.BEFORE_GENERATE_X_START_CHECKPOINT})

for _ in steps_it: pass
fig, (ax1, ax2) = plot_track(opt, target=np.zeros(2), n_points=5, plot_simplex=True, plot_trure=True)
ax2.plot(np.array((-1, 1)), np.array((0, 0)), linestyle='-.')
ax2.plot(np.array((0, 0)), np.array((-1, 1)), linestyle='-.')

margin = 1e-1
xmin = min((*opt.track[:, 0], 0))
xmax = max((*opt.track[:, 0], 0))
ymin = min((*opt.track[:, 1], 0))
ymax = max((*opt.track[:, 1], 0))
ax2.set(xlim=(xmin - margin, xmax + margin), ylim=(ymin - margin, ymax + margin))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[(-0.1311676198933574, 0.7052284831745353),
 (-1.0224395061937117, 0.1138824146744272)]